In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
from IPython.display import display, HTML
from document_parser import get_chunks



In [2]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')
# all-MiniLM-L6-v2
# all-mpnet-base-v2


In [3]:
docs = [
        {
            "url": "https://docs.google.com/document/d/1SxTsELQQafYLNXU7q4dcgR-1K0XbX29i",
            "type": "gdoc"
        },
        {
            "url": "https://github.com/virtual-labs/app-exp-create-web/blob/master/docs/developer-doc.md",
            "type": "md"
        },
        {
            "url": "https://github.com/virtual-labs/app-exp-create-web/blob/master/docs/user-doc.md",
            "type": "md"
        },
        {
            "url": "https://github.com/virtual-labs/app-vlabs-pwa/blob/main/docs/tech_guide.md",
            "type": "md"
        },
        {
            "url": "https://github.com/virtual-labs/app-vlabs-pwa/blob/main/docs/user_manual.md",
            "type": "md"
        },
        {
            "url": "https://docs.google.com/document/d/1lGm88N-Z6fQM6v04k9NZTd-STZ0XYV6YRwIYT1JiSP8/",
            "type": "gdoc"
        }
     ]
data = get_chunks(*docs)

# CMS - Experiments
# Board Game Design


In [4]:
qdrant = QdrantClient(":memory:")

In [5]:
qdrant.recreate_collection(
    collection_name="my_doc",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.DOT
    )
)

True

In [6]:
qdrant.upload_records(
    collection_name="my_doc",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["content"]).tolist(),
            payload=doc["payload"]
        ) for idx, doc in enumerate(data)
    ]
)

In [9]:
queries = [
	"what are future update and releases for content development platform", #0
	"how to modify experiment after creating it", #1
	"getting authentication error in content development platform", #2
	"how do i search experiments in pwa", #3
	"how do i apply filters in vlabs pwa", #4
	"how do i apply filters on search in vlabs pwa", #5
	"what is google's revenue in 2022", #6
	"what is my name", #7
	"Virtual Labs Content Development Platform - Developer Document", #8
	"what was the problem which lead to cms development", #9
	"tell about shauryagarh in board game", #10
]
# argmode, lierate programming
hits = qdrant.search(
	collection_name="my_doc",
	query_vector=encoder.encode(
		queries[10]).tolist(),
    limit=20
    # all-mpnet-base-v2
)
print(len(hits))
table_contents = []
for hit in hits:
	table_contents.append({
		"type": hit.payload["type"], 
		"url": hit.payload["url"],
		"score": hit.score,
		"heading": hit.payload["heading"],
		"document": hit.payload["page_title"],
		"text": hit.payload["text"].split("::")[2].strip(),
	})
df = pd.DataFrame(table_contents)
df["url"] = df["url"].apply(lambda link: f'<a href="{link}">Visit Link</a>')
df["text"] = df["text"].apply(lambda link: link.replace("\n","<br/>"))
display(HTML(df.to_html(escape=False)))


20


,type,url,score,heading,document,text
0,gdoc,Visit Link,0.550236,Narrative,Board Game Design,"The year is 1200 BC. The kingdom of Shauryagarh is in turmoil. The old king is dying, and his four cousins are fighting for the throne.He decides to put them to the test. He summons them to the throne room and tells them that the first one to create five camps by defeating the terrible beasts surrounding Shauryagarh will be the new king.The cousins are excited by the challenge. They set out to conquer the beasts, each with their own strategy.The battle for the throne of Shauryagarh is fierce. The cousins fight against the beasts, each other, and their own inner demons. As the camps start to fall, the stakes get higher and the competition gets more intense.Who will be the first to create five camps? Will the cousins be able to work together, or will they tear each other apart? Dive in and find out!!"
1,gdoc,Visit Link,0.429054,Basic Game Idea,Board Game Design,"Players will start off in the middle-most cell of the board, and will have to defeat monsters in various dungeons to get loot, which will in turn be used to make camps and thereby mark territory. Players can either choose to go alone or work as a team to gain the rewards, but will have to have an element of trust in their opponent if the choose to do the latter."
2,gdoc,Visit Link,0.341671,Game End Condition,Board Game Design,The player to build 5 camps first will be declared the winner.
3,gdoc,Visit Link,0.341157,The Board (Space),Board Game Design,- 9x9 grid of colored cells- 5 is the number of camps required to win the game- 4 players- 17 starred cells (the cells where camps can be built)- Players start at the middle-most cell- Cell Color determines monster difficulty
4,gdoc,Visit Link,0.337144,Time Division (Turn Based),Board Game Design,All Players choose direction and move together and fight their battles in their cells at one turn.
5,gdoc,Visit Link,0.314020,The Theme,Board Game Design,"Trust, the theme of this game is going to be a major aspect of this game, because if the players choose to clear the dungeons as a team (which they will have to at some point), they will have to pick a card of either trust or deception, which will lead to 3 cases:- Both the players choose the trust card - This case will lead to both the players sharing the rewards.- One player chooses trust, but the other chooses deception - In this case, the player choosing deception will get the whole reward.- Both the players choose deception - In this case, both the players will get nothing."
6,gdoc,Visit Link,0.260502,Objects and Attributes,Board Game Design,"- Player- Can move in 4 directions: North, South, East, and West in one turn- Has weapons and loot (starting and ones gained from cells)- Dies if weapons cant beat monster level in cell- Can make camp in camp cells after beating monster in cell- Start Cell- Grants players with a Weak Weapon- Monster Cell- Has 3 types- Easy- Medium- Hard- Has a level associated to it.- When players enter it they have to fight the monster and if their weapon level is lower, they die and lose everything and return to Start Cell or their own camp cell - On winning players can draw loot and weapons from lone stack if alone, team stack if with others- Weapon Card- Has a weapon level number.- Has 3 types- Weak - beats Easy- Sturdy - beats Easy, Medium- Strong - beats Easy, Medium, Hard- A Collection of Weapon Cards can only beat a cell if and only if- ( a ) Collection has atleast one weapon card, that beats the Monster Cell type- ( b ) Sum of all Weapons Card Level is greater than or equal to Monster Cell Level- Loot Card- Acts as currency to build camps- Camp Cell- A special kind of Monster Cell where camps can be built- Only of type Hard- Camp- Can be built using a certain amount of Loot Cards- First Player to 5 camps wins."
7,md,Visit Link,0.153778,Future Scope,Tech Guide : Virtual Labs PWA,"* Filling in the metadata for more experiments so that more experiments are 